Import necessary libraries

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D


Load the dataset

In [8]:
data = pd.read_csv(r"E:\trainingset\archive\trainLabels.csv")

Preprocess the images

In [10]:
def preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (128,128))
    return image

images = []
labels = []

for index, row in data.iterrows():
    image = cv2.imread('E:/trainingset/archive/resized_train/resized_train/' + row['image'] + '.jpeg')
    images.append(preprocess(image))
    labels.append(row['level'])


Split the dataset into training and testing sets

In [11]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

Build the model

In [12]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

Compile and train the model

In [13]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(np.array(X_train), np.array(y_train), batch_size=128, epochs=10, verbose=1)

Epoch 1/10
220/220 [==============================] - 187s 754ms/step - loss: 31.3683 - accuracy: 0.7113
Epoch 2/10
220/220 [==============================] - 167s 758ms/step - loss: 1.3181 - accuracy: 0.7345
Epoch 3/10
220/220 [==============================] - 169s 767ms/step - loss: 1.1767 - accuracy: 0.7346
Epoch 4/10
220/220 [==============================] - 175s 796ms/step - loss: 1.0729 - accuracy: 0.7346
Epoch 5/10
220/220 [==============================] - 192s 873ms/step - loss: 1.0046 - accuracy: 0.7346
Epoch 6/10
220/220 [==============================] - 186s 844ms/step - loss: 0.9595 - accuracy: 0.7346
Epoch 7/10
220/220 [==============================] - 184s 835ms/step - loss: 0.9300 - accuracy: 0.7346
Epoch 8/10
220/220 [==============================] - 179s 815ms/step - loss: 0.9104 - accuracy: 0.7346
Epoch 9/10
220/220 [==============================] - 187s 850ms/step - loss: 0.8973 - accuracy: 0.7346
Epoch 10/10
220/220 [==============================] - 186s 846

Evaluate the model

In [14]:
score = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8815409541130066
Test accuracy: 0.7356959581375122
